# PHINDR3D Results and clustering

Visualize Phindr3D results and perform affinity propagation clustering.

code marked `EDIT HERE` may be edited as needed to get desired results.

In [ ]:
import phindr_clustering as clu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as met

# File Loading

Load saved image feature file and filter as needed.

In [ ]:
#path to csv file: EDIT HERE
filename = r'FILE_PATH'

#filter out images with "Not a number" megavoxel frequencies (happens when no megavoxels found)
Filter_NaN = True 

In [ ]:

image_feature_data_raw = pd.read_csv(filename, index_col=0) 

if Filter_NaN:
    image_feature_data_raw = image_feature_data_raw.dropna()

from IPython.display import display
display(image_feature_data_raw)


# Data filtering

In [ ]:
# Filter dataframe as needed:
#   to filter the dataframe (e.g. to only select orws with specific range of values):
#   set filter_data to True below, change FILTER COLUMN to the desired column, 
#   change FILTER VALUE to the desired value, and check that the operation (==, >, <, <=, >=) is correct.
#   copy and paste the indented filter control lines below to add aditional filtering as needed.
filter_data = False

#rescale texture features to the range [0, 1]
rescale_texture_features = False

# choose dataset to use for clustering: EDIT HERE
# Choices: 
# 'MV' -> megavoxel frequencies, 
# 'text' -> 4 haralick texture features, 
# 'combined' -> both together
datachoice = 'MV'

if filter_data:
    df = image_feature_data_raw

    #filter here 
    #documentation on how to filter Pandas dataframes can be found at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc 

    image_feature_data = df
else:
    image_feature_data = image_feature_data_raw

In [ ]:
#min-max scale all data:
columns = image_feature_data.columns
image_feature_data[columns[4:]] = (image_feature_data[columns[4:]] - image_feature_data[columns[4:]].min())/(image_feature_data[columns[4:]].max()-image_feature_data[columns[4:]].min())
image_feature_data[columns[-4:]] = ((image_feature_data[columns[-4:]] - image_feature_data[columns[-4:]].min())/(image_feature_data[columns[-4:]].max()-image_feature_data[columns[-4:]].min()))

#get misc values
num_images_kept = image_feature_data.shape[0]
print(f'\nNumber of images: {num_images_kept}\n')
imageIDs = np.array(image_feature_data['ImageID'], dtype='object')
uniqueIDs = np.unique(imageIDs)
treatments = np.array(image_feature_data['treatment'], dtype='object')
Utreatments = np.unique(treatments)
numMVperImg = np.array(image_feature_data['numMV'])

numMV = image_feature_data.shape[1]-3
mv_cols = columns[3:-4] #all columns corresponding to megavoxel categories #should usually be -5 since contrast is still included here.
MV_freqs = image_feature_data[mv_cols].to_numpy()
texture_cols = columns[-4:]
# if rescale_texture_features:
#     for i in range(len(texture_cols)):
#         image_feature_data[texture_cols[i]] = clu.rescale(image_feature_data[texture_cols[i]], newmin=0, newmax=1)
texture_data = image_feature_data[texture_cols].to_numpy()
combined_cols = columns[3:]
combined_data = image_feature_data[combined_cols].to_numpy()

y = imageIDs
z = treatments
if datachoice.lower() == 'mv':
    X = MV_freqs
elif datachoice.lower() == 'text':
    X = texture_data
elif datachoice.lower() == 'combined':
    X = combined_data
else:
    print('ERROR invalid data set choice.')
print('Dataset shape:', X.shape, '\n')

print('Treatments found:')
print(Utreatments)

if len(Utreatments) > 10:
    import matplotlib as mpl
    colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, 20))
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors) 

from IPython.display import display
# display(image_feature_data)
display(image_feature_data.describe())

# Sammon Mapping
Make sammon map of Phindr3D data


In [ ]:
#plot parameters: EDIT HERE
title = 'Sammon map'
xlabel = 'Axis 1'
ylabel = 'Axis 2'

S, E = clu.sammon(X, 2)
print(S.shape)

fig, ax = plt.subplots(figsize=(10,10))
for treat in Utreatments:
    ax.scatter(S[z==treat, 0], S[z==treat, 1], label=treat)
ax.legend()
ax.set_title(title)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
plt.show()

# PCA plot
Make PCA plot of phindr3D results. currently uses kernel PCA with a variable function since it seems to get best results here. 

See https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html for more information on types of PCA availble.

In [ ]:
#PCA kernel function: EDIT HERE
#set as 'linear' for linear PCA, 'rbf' for gaussian kernel, 
#'sigmoid' for sigmoid kernel, 
#'cosine' for cosine kernel
func = 'rbf'

#plot parameters: EDIT HERE
title = 'PCA plot'
xlabel = 'PCA 1'
ylabel = 'PCA 2'

#makes plot 
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_show = sc.fit_transform(X)
pca = KernelPCA(n_components=2, kernel=func) 
P = pca.fit(X_show).transform(X_show)

plt.figure(figsize=(10,10))
for treat in Utreatments:
    plt.scatter(P[z==treat, 0], P[z==treat, 1], label=treat)
plt.legend()
plt.title(title)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()


# t-SNE

Make t-SNE plot of phindr3D data. 

t-SNE is not a deterministic method, results may vary between different runs. Check https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html for additional parameters to use


In [ ]:
#plot parameters: EDIT HERE
title = 't-SNE plot'
xlabel = 'Axis 1'
ylabel = 'Axis 2'


from sklearn.manifold import TSNE
T = TSNE(n_components=2, init='pca', learning_rate='auto').fit_transform(X) #can edit here from tsne documentation.

plt.figure(figsize=(10,10))
for treat in Utreatments:
    plt.scatter(T[z==treat, 0], T[z==treat, 1], label=treat)
plt.legend()
plt.title(title)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()

# AP clustering
cluster phindr3D results into k clusters using affinity propagation. This process may take a few minutes.

In [ ]:
#clustering parameters: EDIT HERE
Nclusters = 7 #number of clusters to try to hit
percent_dev = 1 #percentage by which final number of clusters may deviate from Nclusters


#performs clustering
C = clu.clsIn(X) #make similarity matrix
idx, netsim, dpsim, expref, pref = clu.apclusterK(C.S, Nclusters, prc=percent_dev)
clusters, counts = np.unique(idx, return_counts=True)
print('\n')
for i in range(len(clusters)):
    print(f'cluster{i+1}: {counts[i]} counts')

# Cluster rating
Used mutual information scores to rate quality of clustering

In [ ]:
z#treatments
idx#which cluster
treatlabels = np.zeros(z.shape)
for i, t in enumerate(z):
    treatlabels[z==t] = i+1

print('Mutual information:', met.mutual_info_score(treatlabels, idx, ))
print('Normalized mutual information:', met.normalized_mutual_info_score(treatlabels, idx))
print('Adjusted mutual information:', met.adjusted_mutual_info_score(treatlabels, idx))

# Cluster visualization
- Show clusters in reduced dimensionality plot
- cluster pie charts
- cluster heatmaps

In [ ]:
#visualize clusters

#choose type of mapping: EDIT HERE
#options: 'sammon', 'pca', 'tsne'
map_type = 'sammon'

#plot parameters: EDIT HERE
title = 'Cluster map'
xlabel = 'Axis 1'
ylabel = 'Axis 2'


if map_type.lower() == 'sammon':
    show = S
elif map_type.lower() == 'tsne':
    show = T
elif map_type.lower() == 'pca':
    show = P
#makes plot
plt.figure(figsize=(10,10))
for i in range(len(clusters)):
    plt.scatter(show[idx==clusters[i], 0], show[idx==clusters[i], 1], label=f'Cluster {i+1}')
plt.legend()
plt.title(title)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()

In [ ]:
#make heatmap of cluster distribution for each treatment
#plot parameters: EDIT HERE
title = 'Cluster heatmap'
xlabel = 'Cluster'
ylabel = 'Treatment'

# EDIT HERE: set ltreatment labels in desired output order
orderedTreatments = [ 'example1', 'example2', 'example3', 'example4']

map = np.zeros((len(orderedTreatments), len(clusters)))
for i, treat in enumerate(orderedTreatments):
    for j in range(len(clusters)):
        map[i, j] = np.sum(np.logical_and(treatments==treat, idx==clusters[j]))

#normalize the cluster counts for each treatment
row_sum = np.sum(map, axis=1)
map = map / row_sum[:, np.newaxis]
map_bad = np.logical_not(np.isfinite(map)) #clean up any potential divide by 0 errors
map[map_bad] = 0
    
#make plot
plt.figure(figsize=(12,8))
plt.title(title)
plt.imshow(map, cmap='plasma', aspect='auto')
plt.xticks(ticks=[i for i in range(0, len(clusters))], labels=[f'{i}' for i in range(1, len(clusters)+1)])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.yticks(ticks=[i for i in range(len(orderedTreatments))], labels=orderedTreatments)
plt.colorbar()
plt.show()

In [ ]:
#make pie charts for each cluster
#pie chart making is all automated, only need to run this cell.


def percent(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.1f}%".format(pct, absolute)

z = treatments

for i in range(len(clusters)): 
    counts = np.zeros(len(Utreatments))
    for j, treat in enumerate(Utreatments):
        counts[j] = np.sum(np.logical_and(z==treat, idx==clusters[i]))
    labels = Utreatments
    fig, ax = plt.subplots(figsize=(15,8))
    title = f'Cluster {i+1} composition ({round(np.sum(counts))} images)'
    ax.set_title(title)
    wedges, texts, autotexts = ax.pie(counts, labels=labels, autopct= lambda pct:percent(pct, counts))
    ax.legend(wedges, labels, loc='center right', bbox_to_anchor=(0.8, 0.5, 0.5, 0.5))
    fig.set_facecolor('white')

plt.show()


In [ ]:
#make heatmap of cluster distribution for each treatment
#plot parameters: EDIT HERE
title = 'Neuron cluster distribution'
xlabel = 'Cluster'
ylabel = 'Treatment'

fclusters = np.delete(clusters, 2)

map = np.zeros((len(Utreatments), len(fclusters)))
for i, treat in enumerate(Utreatments):
    for j in range(len(fclusters)):
        map[i, j] = np.sum(np.logical_and(treatments==treat, idx==fclusters[j]))

#normalize the cluster counts for each treatment
row_sum = np.sum(map, axis=1)
map = map / row_sum[:, np.newaxis]
map_bad = np.logical_not(np.isfinite(map))
map[map_bad] = 0

    
#make plot
plt.figure(figsize=(8,8))
plt.title(title)
plt.imshow(map, cmap='jet', aspect='auto')
plt.xticks(ticks=[i for i in range(0, len(fclusters))], labels=[1,2,4,5])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.yticks(ticks=[i for i in range(len(Utreatments))], labels=Utreatments)
plt.colorbar()
plt.show()